In [1]:
% matplotlib inline

/Users/kei/penv2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.externals import joblib
import time

In [2]:
advertiserlist_path = '../data/adtech_intern_advertiserlist.csv'
log_path = '../data/intern_samplelog.csv'

In [3]:
advertiserlist = pd.read_csv(advertiserlist_path)
log_original = pd.read_csv(log_path)
log = log_original

In [4]:
log.loc[:,'os'][log.os=='iOS'] = 0
log.loc[:,'os'][log.os=='Android'] = 1

/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
def hashnum(modnum, data):
    mod = 10 ** modnum
    return hash(data) % mod

In [8]:
logEachAd = [log[log.advertiser==ad].drop('advertiser', axis=1).fillna(value=0) for ad in xrange(1, 11)]
input_type='string'
param = {'max_depth':6, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 11
modnum = 2
n_folds = 10

for ind, eachlog in enumerate(logEachAd):
    print '----------------- Advertiser %d -------------------' %(ind+1)
    log_data = eachlog.drop('click', axis=1)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    print xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)
    xgb_model = xgb.train(param, dlog, num_round)
    model_path = '../models/Ad%d_xgb.pkl'%(ind+1)
    joblib.dump(xgb_model, model_path)

----------------- Advertiser 1 -------------------
    test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.650996      0.006061        0.654391       0.001684
1        0.666864      0.003790        0.669889       0.002226
2        0.669218      0.004200        0.673909       0.001634
3        0.669182      0.004188        0.678239       0.001068
4        0.668262      0.004392        0.680646       0.001081
5        0.666563      0.004622        0.683008       0.001066
6        0.666160      0.004403        0.684716       0.000972
7        0.665672      0.004625        0.686069       0.000936
8        0.664971      0.004507        0.687422       0.000763
9        0.664419      0.004287        0.688756       0.000778
10       0.663666      0.004553        0.689983       0.000814
----------------- Advertiser 2 -------------------
    test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.554664      0.007393        0.557706       0.006277
1        0.56911

In [9]:
logEachAd = [log[log.advertiser==ad].drop('advertiser', axis=1).fillna(value=0) for ad in xrange(1, 11)]
input_type='string'
param = {'max_depth':6, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 11
modnum = 3
n_folds = 10

for ind, eachlog in enumerate(logEachAd):
    print '----------------- Advertiser %d -------------------' %(ind+1)
    log_data = eachlog.drop('click', axis=1)
    log_target = eachlog.click
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os, log_data.floor_price]
    dlog = xgb.DMatrix(log_data,label=log_target)
    print xgb.cv(param, dlog, num_round, nfold=n_folds, metrics={'auc'}, seed = 0)
    xgb_model = xgb.train(param, dlog, num_round)
    model_path = '../models/Ad%d_xgb.pkl'%(ind+1)
    joblib.dump(xgb_model, model_path)

----------------- Advertiser 1 -------------------
    test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.650861      0.006171        0.654020       0.001627
1        0.666785      0.003373        0.669649       0.002339
2        0.668230      0.003480        0.673692       0.001721
3        0.669490      0.004180        0.679061       0.001144
4        0.668077      0.004019        0.681845       0.000812
5        0.667703      0.004041        0.684197       0.000897
6        0.666623      0.004009        0.686149       0.000798
7        0.665544      0.004644        0.688082       0.000857
8        0.664584      0.004813        0.689793       0.000932
9        0.664403      0.004956        0.691464       0.001073
10       0.663836      0.005033        0.692935       0.001171
----------------- Advertiser 2 -------------------
    test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
0        0.554345      0.006989        0.558015       0.006110
1        0.56788

In [63]:
xgb_models = [joblib.load('../models/Ad%d_xgb.pkl'%ind) for ind in xrange(1,11)]

In [51]:
test_samples = log.drop(['click', 'advertiser'], axis=1).loc[:10000].fillna(value=0)
start_time = time.time()

for test_sample in test_samples.as_matrix():
    if test_sample[3] == 'iOS':
        test_sample = xgb.DMatrix([hashnum(modnum, test_sample[0]), hashnum(modnum, test_sample[2]), 0])
    else:
        test_sample = xgb.DMatrix([hashnum(modnum, test_sample[0]), hashnum(modnum, test_sample[2]), 1])
    [float(model.predict(test_sample))for model in xgb_models]
print time.time() - start_time

25.8333959579


In [80]:
sample_dict = {
    'id' : "ididid",
    'floorPrice' : 1000,
    'site' : "medianame",
    'device' : "device",
    'user' : "a9102910201",
    'test' : 0
}

In [84]:
if sample_dict['floorPrice'] == 'nan':
    sample_dict['floorPrice'] = 0
if sample_dict['device'] == 'iOS':
        test_sample = xgb.DMatrix([hashnum(modnum, sample_dict['site']), hashnum(modnum, sample_dict['user']), 0,  sample_dict['floorPrice']])
else:
    test_sample = xgb.DMatrix([hashnum(modnum, sample_dict['site']), hashnum(modnum, sample_dict['user']), 1,  sample_dict['floorPrice']])
[float(model.predict(test_sample))for model in xgb_models]

[0.21913763880729675,
 0.14578142762184143,
 0.18474996089935303,
 0.19934862852096558,
 0.7245240211486816,
 0.24230889976024628,
 0.6336053609848022,
 0.22833609580993652,
 0.19325865805149078,
 0.8674460053443909]